# Описание задачи

*market_file.csv* - таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.

`id` — номер покупателя в корпоративной базе данных.

`Покупательская активность` — рассчитанный класс покупательской активности (целевой признак): «снизилась» или «прежний уровень».

`Тип сервиса` — уровень сервиса, например «премиум» и «стандарт».

`Разрешить сообщать` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель.

`Маркет_актив_6_мес` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента.

`Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце.

`Длительность` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте.

`Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев.

`Популярная_категория` — самая популярная категория товаров у покупателя за последние 6 месяцев.

`Средний_просмотр_категорий_за_визит` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца.

`Неоплаченные_продукты_штук_квартал` — общее число неоплаченных товаров в корзине за последние 3 месяца.

`Ошибка_сервиса` — число сбоев, которые коснулись покупателя во время посещения сайта.

`Страниц_за_визит` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.


*market_money.csv* - таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.

`id` — номер покупателя в корпоративной базе данных.

`Период` — название периода, во время которого зафиксирована выручка. Например, 'текущий_месяц' или 'предыдущий_месяц'.

`Выручка` — сумма выручки за период.


*market_time.csv* - таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.

`id` — номер покупателя в корпоративной базе данных.

`Период` — название периода, во время которого зафиксировано общее время.

`минут` — значение времени, проведённого на сайте, в минутах.


*money.csv* - таблица с данными о среднемесячной прибыли продавца за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

`id` — номер покупателя в корпоративной базе данных.

`Прибыль` — значение прибыли.

In [ ]:
#!pip install -q scikit-learn==1.5.0
#!pip install --upgrade shap matplotlib
#!pip install shap
#!pip install phik
#!pip install yellowbrick
#!pip install --upgrade seaborn

In [ ]:
from scipy import stats

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import seaborn as sns
import shap

from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, recall_score, roc_auc_score
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler,
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## Загрузка данных

In [ ]:
mfile=pd.read_csv('/datasets/market_file.csv')
mmoney=pd.read_csv('/datasets/market_money.csv')
mtime=pd.read_csv('/datasets/market_time.csv')
money=pd.read_csv('/datasets/money.csv', sep=';')

In [ ]:
display(mfile)

In [ ]:
display(mmoney)

In [ ]:
display(mtime)

In [ ]:
display(money)

In [ ]:
mfile.info()

In [ ]:
mmoney.info()

In [ ]:
mtime.info()

In [ ]:
money.info()

Некоторые названия столбцов надо привести к общему виду. Пропуски отсутствуют. Типы данных соответствуют описанию столбцов. В файле *money* в столбце `Прибыль` тип данных не соответствует.

## Предобработка данных

Изменим названия столбцов:

In [ ]:
mfile.columns = mfile.columns.str.replace('Покупательская активность', 
                                          'Покупательская_активность').str.replace(
    'Тип сервиса', 'Тип_сервиса').str.replace('Разрешить сообщать', 'Разрешить_сообщать')

In [ ]:
mfile.columns = mfile.columns.str.lower()
mmoney.columns = mmoney.columns.str.lower()
mtime.columns = mtime.columns.str.lower()
money.columns = money.columns.str.lower()

Найдем дубликаты:

In [ ]:
display(mfile.duplicated().sum())
display(mmoney.duplicated().sum())
display(mtime.duplicated().sum())
display(money.duplicated().sum())

Проверим таблицы на наличие неявных дубликатов. Для этого создадим две функции: функция *categorize_columns* разделяет столбцы на категориальные и числовые без учета столбца `id`, функция *uniques* выводит уникальные значения категориального столбца.

In [ ]:
def categorize_columns(df):
    df = df.drop('id', axis=1, errors='ignore')
    num_cols = [] 
    cat_cols = []  
    for col in df.columns:
        if df.dtypes[col] == 'O':
            cat_cols.append(col)
        else:
            num_cols.append(col) 
    return num_cols, cat_cols

In [ ]:
def uniques(df):
    for col in categorize_columns(df)[1]:
        unique_values = df[col].unique()
        print(f"Уникальные значения в столбце '{col}': {unique_values}")

In [ ]:
uniques(mfile)

In [ ]:
uniques(mmoney)

In [ ]:
uniques(mtime)

Устраним неявные дубликаты и исправим ошибки:

In [ ]:
mfile['тип_сервиса'] = mfile['тип_сервиса'].replace('стандартт', 'стандарт')
mtime['период'] = mtime['период'].replace('предыдцщий_месяц', 'предыдущий_месяц')

Заменим тип данных в *money* на правильный:

In [ ]:
money['прибыль'] = money['прибыль'].replace(',','.',regex=True).astype('float64')

In [ ]:
mfile.describe(include='all')

In [ ]:
mmoney.describe(include='all')

In [ ]:
mtime.describe(include='all')

In [ ]:
money.describe(include='all')

В датафремейме *mmoney* есть выброс. Проверим этот выброс, посмотрим на траты пользователя ранее, а также на его характеристику из основного файла:

In [ ]:
mmoney.query('выручка > 10000')

In [ ]:
mmoney.query('id == 215380')

In [ ]:
mfile.query('id == 215380')

Так как его покупательская активность снизилась, заменим выброс значением 5000:

In [ ]:
mmoney.loc[mmoney['выручка'] == 106862.2, 'выручка'] = 5000

Исправлено несколько значений и названий столбцов. В одном столбце заменен тип данных.

## Исследовательский анализ данных

Статистический анализ:

Функция для построения гистрограм для числовых столбцов:

In [ ]:
def plot_hist(df):
    for col in categorize_columns(df)[0]:
        plt.hist(df[col])
        plt.xlabel(col);
        plt.ylabel('Количество');
        plt.title(f'Гистограмма по столбцу "{col}"')
        plt.grid(True)
        plt.show()

Функция для построения столбчатых диаграмм по категориальным столбцам:

In [ ]:
def plot_bar(df):
    for col in categorize_columns(df)[1]:
        df[col].value_counts().plot(kind='bar', figsize=(10, 5))
        plt.title(f'Столбчатая диаграмма по столбцу "{col}"')
        plt.ylabel('Количество')
        plt.xlabel(col)
        plt.grid(True)
        plt.show()

Датафрейм *mfile*:

In [ ]:
fig, axes = plt.subplots(len(categorize_columns(mfile)[0]), 1, figsize=(8, 30))
for i, col in enumerate(categorize_columns(mfile)[0]):
    sns.histplot(data=mfile, x=col, hue='покупательская_активность', kde=True, ax=axes[i])
    axes[i].set_title(f'Распределение {col} по таргету')

plt.tight_layout()
plt.show()

Некоторые количественные cтолбцы явно носят дискретный характер. Используем *pie* и *bar* графики.

In [ ]:
df1 = mfile['маркет_актив_тек_мес'].value_counts()
plt.figure(figsize=(6, 6))
plt.pie(df1, labels=df1.index, autopct='%1.2f%%')
plt.title('Доля маркетинговых коммуникаций в текущем месяце')
plt.show()

In [ ]:
mfile['средний_просмотр_категорий_за_визит'].value_counts().plot(kind='bar', figsize=(10, 5))
plt.title('Средний просмотр категорий за визит')
plt.ylabel('Количество просмотров')
plt.xlabel('Количество просмотренных категорий')
plt.grid(True)
plt.show()

In [ ]:
#Сделаем большие графики, чтоб влезли названия категорий
fig, axes = plt.subplots(len(categorize_columns(mfile)[1]), 1, figsize=(15, 25))
for i, col in enumerate(categorize_columns(mfile)[1]):
    sns.countplot(data=mfile, x=col, hue='покупательская_активность', ax=axes[i])
    axes[i].set_title(f'Распределение {col} по таргету')

plt.tight_layout()
plt.show()

Датафрейм *mmoney*:

In [ ]:
plot_hist(mmoney)

Датафрейм *mtime*:

In [ ]:
plot_hist(mtime)

Датафрейм *money*:

In [ ]:
plot_hist(money)

Построим ящики с усами:

In [ ]:
numeric_cols = mfile.drop('id', axis = 1).select_dtypes(include=['number']).columns 
mfile[numeric_cols].plot(kind='box', subplots=True, layout=(5, 3), figsize=(15, 20))
plt.tight_layout()
plt.show()

In [ ]:
numeric_cols = mfile.select_dtypes(include=['number']).columns

fig, axes = plt.subplots(len(numeric_cols), 1, figsize=(8, len(numeric_cols) * 3))
for i, col in enumerate(numeric_cols):
    sns.histplot(data=mfile, x=col, hue='покупательская_активность', kde=True, ax=axes[i])
    axes[i].set_title(f'Распределение {col} по таргету')

plt.tight_layout()
plt.show()

Есть группа активных пользователей с высокой выручкой (*выручка_тек_мес*, *выручка_пред_мес*);
Пользователи по-разному проводят время на сайте – можно сегментировать их по времени взаимодействия и страницам за визит;
Большинство покупателей мало участвуют в акциях, но есть небольшая очень активная группа.

Целевой признак `покупательская_активность` несбалансирован. Все категориальные столбцы являются бинарными за исключением `популярная_категория`. Столбец `Маркет_актив_тек_мес` явно категориальный. Большинство числовых столбцов распределены нормально.

Надем покупателей с активностью за последние три месяца:

In [ ]:
passive_id = mmoney.query('выручка == 0')['id'].unique()
active_customers = mfile[~mfile['id'].isin(passive_id)]
display(active_customers)

Удалили трех неактивных пользователей.

## Объединение таблиц

In [ ]:
mmoney = mmoney.pivot(index='id', columns='период', values='выручка').reset_index()
mmoney.columns = mmoney.columns.str.replace(
    'предыдущий_месяц','выручка_пред_мес').str.replace(
    'препредыдущий_месяц','выручка_препред_мес').str.replace(
'текущий_месяц','выручка_тек_мес')
mfile = mfile.merge(mmoney, on='id', how='left')


In [ ]:
mtime = mtime.pivot(index='id', columns='период', values='минут').reset_index()
mtime.columns = mtime.columns.str.replace(
    'предыдущий_месяц','время_пред_мес').str.replace(
'текущий_месяц','время_тек_мес')
mfile = mfile.merge(mtime, on='id', how='left')


Удалим неактивных покупателей:

In [ ]:
active_customers = mfile[~mfile['id'].isin(passive_id)]

## Корреляционный анализ

In [ ]:
active_customers.drop('id', axis = 1).corr()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(active_customers.drop('id', axis = 1).corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Корреляционная матрица количественных признаков")
plt.show()

Построим корреляцию Спирмена:

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(active_customers.drop('id', axis = 1).corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица количественных признаков по Спирмену")
plt.show()

`выручка_пред_мес` и `выручка_тек_мес` — дублирующий признак, возможно, стоит оставить только один.

Выведем матрицы корреляций *Phi_K*, разделив данные по целевому признаку:

Функция для визуализации матриц корреляций:

In [ ]:
def plot_corr(corr_matrix, title):
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
    plt.title(title)
    plt.show()

In [ ]:
# Подготовим данные. Столбец "id" неинформативен, столбец "покупательская_активность" будет вызывать ошибку
to_drop = ['покупательская_активность', 'id']

decrease_act = active_customers[active_customers['покупательская_активность'] == 'Снизилась'].drop(to_drop, axis = 1)
stable_act = active_customers[active_customers['покупательская_активность'] == 'Прежний уровень'].drop(to_drop, axis = 1)

# Вычисляем матрицы Phi_K
corr_matrix_decreasing = decrease_act.phik_matrix()
corr_matrix_stable = stable_act.phik_matrix()

plot_corr(corr_matrix_decreasing, "Phi_K корреляция (Снижающаяся активность)");
plot_corr(corr_matrix_stable, "Phi_K корреляция (Не изменяющаяся активность)");

Выведем матрицу разницы корреляций. Вычтя Phi_K-корреляцию "Снижающаяся активность" из "Не изменяющаяся активность", получим матрицу, показывающую, как изменилась связь между переменными при снижении активности клиентов.

Если разность положительная, у стабильных клиентов эта связь сильнее, чем у снижающихся.

Если разность отрицательная, у снижающихся клиентов эта связь сильнее, чем у стабильных.

In [ ]:
plot_corr(corr_matrix_stable - corr_matrix_decreasing, "Phi_K корреляция (Разность стабильной и снижающейся активностей)");

У снижающихся клиентов уменьшилась связь между длительностью сессий (`страниц за визит`) и другими показателями. Это может значить, что долгие сессии не обязательно ведут к покупкам. <br>
`Длительность` теряет связь с другими признаками. <br>
Маркетинговая активность слабее влияет на снижающихся пользователей. <br>
Время проведенное на сайте снижается при снижении покупательской активности. <br>

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(data=active_customers, x='выручка_пред_мес', y='выручка_тек_мес', hue='покупательская_активность')
plt.title('')
plt.xlabel('Выручка за прошлый месяц')
plt.ylabel('Выручка за текущий месяц')
plt.show()

In [ ]:
delta = active_customers['выручка_тек_мес'] - active_customers['выручка_пред_мес']
plt.figure(figsize=(12, 10))
sns.scatterplot(data=active_customers, x='выручка_пред_мес', y=delta, hue='покупательская_активность')
plt.title('Разница выручек в зависимости от предыдущей выручки')
plt.xlabel('Выручка за прошлый месяц')
plt.ylabel('Разница выручек(текущий - прошлый)')
plt.show()

Есть клиенты с повторяющимися расходами: <br>

Это может быть связано с подписками, регулярными заказами или фиксированным бюджетом. <br>

Скорее всего клиенту разделены по какому-то признаку (например, бизнес / частные клиенты, подписки / разовые покупки). <br>

В этих группах можно будет точно прогнозировать выручку. <br>

## Использование пайплайнов

Напишем пайплайн:

In [ ]:
#Подготовка данных:
X = active_customers.drop(['покупательская_активность','id'], axis = 1)
y = active_customers['покупательская_активность']
y = y.map({'Снизилась': 1, 'Прежний уровень': 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, \
test_size=0.25, \
random_state=1, stratify=y)

In [ ]:
#Разбиение данных на типы:
cat_col_names = X_train.select_dtypes(exclude='number').drop('тип_сервиса',axis=1).columns.tolist()
ord_col_names = ['тип_сервиса']
num_col_names = X_train.select_dtypes(include='number').columns.tolist()

In [ ]:
#Пайплайн:
ord_pipe = Pipeline([
    ('ord', OrdinalEncoder(categories=[['стандарт', 'премиум']]))
])

cat_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

num_pipe = Pipeline([('scaler', StandardScaler())])

data_preprocessor = ColumnTransformer([('cat', cat_pipe, cat_col_names),
                                       ('num', num_pipe, num_col_names),
                                       ('ord', ord_pipe, ord_col_names)])

pipe_final = Pipeline([('preprocessor', data_preprocessor),
                      ('models',LogisticRegression(random_state=1))])

param_grid = [
    {
        'models':[LogisticRegression(random_state=1,
                                     solver='liblinear',
                                     penalty='l1')],
        'models__C': range(1,5),
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models':[KNeighborsClassifier()],
        'models__n_neighbors': range(2,5),
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models': [DecisionTreeClassifier(random_state=1)],
        'models__max_depth': range(2,5),
        'models__max_features': range(2,5),
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models': [SVC(random_state=1, probability = True)],
        'models__C': [0.01, 0.1, 1, 10, 100],
        'models__kernel': ['linear', 'rbf', 'poly'],
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    }
]

rand_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    random_state=1,
    n_jobs=-1,
    n_iter = 10
)

#grid = GridSearchCV(
#    pipe_final, 
#    param_grid=param_grid, 
#    cv=5, 
#    scoring='roc_auc', 
#    n_jobs=-1
#)

#grid.fit(X_train, y_train)
rand_search.fit(X_train, y_train)

y_test_pred = rand_search.predict(X_test)
y_test_pred_proba = rand_search.predict_proba(X_test)[:, 1]  

#y_test_pred_grid = grid.predict(X_test)

print('Лучшая модель и её параметры:\n\n', rand_search.best_estimator_)
print(f'Средняя ROC-AUC на кросс-валидации:', rand_search.best_score_)
print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred_proba)}')
print(f'Метрика f1_score на тестовой выборке: {f1_score(y_test, y_test_pred)}')
print(f'Метрика recall_score на тестовой выборке: {recall_score(y_test, y_test_pred)}')

#print('Лучшая модель и её параметры:\n\n', grid.best_estimator_)
#print ('Метрика лучшей модели на тренировочной выборке:', grid.best_score_)
#print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred_grid)}')

Используем метрику Recall: бизнесу надо удержать клиентов, поэтому нам надо ловить максимум пользователей, у которых реально снижается активность (но можем ошибочно пометить стабильных).

Результаты работы GridSearch:

    ('models', SVC(C=0.1, random_state=1))])
    Метрика лучшей модели на тренировочной выборке: 0.9076807832626013
    Метрика ROC-AUC на тестовой выборке: 0.8746589632482747

## Анализ важности признаков

Построим график убывания средних SHAP‑значений:

In [ ]:
#Обучим данные, взяв из пайплана препроцессор:
X_transformed = rand_search.best_estimator_.named_steps['preprocessor'].transform(X_test)

In [ ]:
#Возьмем из пайплана названия признаков:
feature_names = rand_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()

In [ ]:
#Это работает очень криво, очень долго ковырялся 

explainer = shap.Explainer(rand_search.best_estimator_.named_steps['models'], X_transformed)
shap_values = explainer(X_transformed)
#fig, ax = plt.subplots(figsize=(20, 20))  
shap.summary_plot(shap_values, X_transformed, feature_names=feature_names, show=False)
#plt.show() 
#shap.plots.beeswarm(shap_values, X_transformed, feature_names=feature_names, show=False)
#shap.summary_plot(shap_values, X_transformed, feature_names=feature_names, show=False)
plt.gcf().set_size_inches(20, 12)  # Некоторые названия слишком длинные и из-за них график сужается
plt.tight_layout()
plt.show()

Наиболее важные признаки:

`num_страниц_за_визит`<br>
`num_время_пред_мес`<br>
`num_время_тек_мес`<br>
`num_средний_просмотр_категорий_за_визит`<br>
`num_акционные_покупки`<br>

Наименее важные признаки:

`num_маркет_актив_тек_мес`<br>
`num_ошибка_сервиса`<br>
и все признаки из `популярная_категория`

## Сегментация покупателей

Группа клиентов, которые покупают только технику, то есть товары с длинным жизненным циклом.

Cовершают редкие, но дорогие покупки.<br>
Не заинтересованы в регулярных покупках мелких товаров.<br>
Тщательно выбирают товар, сравнивают цены и читают отзывы.<br>

Причины низкой частоты покупок:
Долгий срок службы.<br>
Высокая стоимость.<br>
Клиенты могут долго искать эту технику, ждать скидок, изучать отзывы, смотреть вторичный рынок и тд.<br>

Посмотрим на интересующие нас признаки:<br>
`num_страниц_за_визит`<br>
`num_время_пред_мес`<br>
`num_время_тек_мес`<br>
`num_средний_просмотр_категорий_за_визит`<br>
`num_акционные_покупки`<br>

In [ ]:
top5 = ['страниц_за_визит',
        'время_пред_мес',
        'время_тек_мес',
        'средний_просмотр_категорий_за_визит','акционные_покупки'
       ]

In [ ]:
plot_hist(active_customers.query(
    'популярная_категория == "Мелкая бытовая техника и электроника" and покупательская_активность == "Снизилась"')[top5])


Рассмотрим интересующие нас признаки:<br>
`num_страниц_за_визит`<br>
`num_время_пред_мес`<br>
`num_время_тек_мес`<br>
`num_средний_просмотр_категорий_за_визит`<br>
`num_акционные_покупки`<br>

Пользователи мало проводят времени на сайте, просматривают мало категорий, посещение страниц распределено равномерно. Все эти признаки надо увеличить. Акционные покупки согласно SHAP анализу повышают шанс понижения покупательской активности, но с этим ничего не сделаешь.

Для увеличения покупательской активности в этой категории можно сделать следующее:
специальные предложения по продаже дополнительных акссесуаров к технике; <br>
подписки на дополнительную страховку, гарантию, обслуживание; <br>
услуги по установке, обучению, ремонту, настройке; <br>
trade-in старой техники на новую; <br>
рассрочка; <br>

Возьмем сегмент клиентов с высокой вероятностью снижения покупательской активности (больше 0.8) и наиболее высокой прибыльностью (больше медианы прибыли в текущем месяце и предыдущем месяце):

In [ ]:
#Pandas требует делать это через copy.
active_customers = active_customers.copy()
active_customers.loc[:, 'proba'] = rand_search.predict_proba(active_customers)[:, 1]
active_customers['сум_выручка'] = active_customers['выручка_пред_мес'] + active_customers['выручка_тек_мес']
#Выделим строки с нашим сегментом с помощью нового столбца:
active_customers.loc[active_customers.query('proba > 0.8 and сум_выручка > @active_customers["сум_выручка"].median()').index, 'сегмент'] = 'Целевой сегмент'
active_customers['сегмент'].fillna('Остальные', inplace=True)

active_customers

In [ ]:
sns.histplot(data=active_customers, x='сум_выручка', hue='сегмент', kde=True)
plt.title('Распределение прибыльности клиентов')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
sns.countplot(data=active_customers, x='популярная_категория', hue='сегмент')
plt.title('Распределение прибыльности клиентов')
plt.show()

In [ ]:
cols = ['страниц_за_визит', 'время_пред_мес', 'время_тек_мес', 'средний_просмотр_категорий_за_визит', 'акционные_покупки', 'маркет_актив_6_мес']
fig, axes = plt.subplots(len(cols), 1, figsize=(6,22))
for i, col in enumerate(cols):
    sns.histplot(data=active_customers, y=col,x='proba', hue='сегмент', ax=axes[i])
    axes[i].set_title(f'Распределение {col} по таргету')

plt.tight_layout()
plt.show()


Пользователи в сегменте покупают в основном косметику и аксессуары, кухонную посуду и детские товары. Сегмент меньше времени проводит на сайте, но все еще пользуется акциями. Также сократилось количество маркетинговых взаимодействий с клиентом. Исходя из этого, можно проводить дальнейший анализ 

## Вывод

1. Некоторые названия столбцов надо привести к общему виду. Пропуски отсутствуют. Типы данных соответствуют описанию столбцов. В файле *money* в столбце `Прибыль` тип данных не соответствует.
2. Исправлено несколько значений и названий столбцов. В одном столбце заменен тип данных.
3. Целевой признак `покупательская_активность` несбалансирован. Все категориальные столбцы являются бинарными за исключением `популярная_категория`. Столбец `Маркет_актив_тек_мес` явно категориальный. Большинство числовых столбцов распределены нормально. Было удалено 3 неактивных пользователя
4. У снижающихся клиентов уменьшилась связь между длительностью сессий (`страниц за визит`) и другими показателями. Это может значить, что долгие сессии не обязательно ведут к покупкам. <br>
    `Длительность` теряет связь с другими признаками. <br>
    Маркетинговая активность слабее влияет на снижающихся пользователей. <br>
    Время проведенное на сайте снижается при снижении покупательской активности. <br>
    Есть клиенты с повторяющимися расходами: <br>
    Это может быть связано с подписками, регулярными заказами или фиксированным бюджетом. <br>
    Скорее всего клиенту разделены по какому-то признаку (например, бизнес / частные клиенты, подписки / разовые покупки). <br>
    В этих группах можно будет точно прогнозировать выручку. <br>
5. Результаты работы GridSearch: лучшая модель *SVC* (C=0.1, random_state=1) <br>
    Средняя ROC-AUC на кросс-валидации: 0.92 <br>
    Метрика ROC-AUC на тестовой выборке: 0.89 <br>
    Метрика f1_score на тестовой выборке: 0.82 <br>
    Метрика recall_score на тестовой выборке: 0.77 <br>
6. Наиболее важные признаки:

`num_страниц_за_визит`<br>
`num_время_пред_мес`<br>
`num_время_тек_мес`<br>
`num_средний_просмотр_категорий_за_визит`<br>
`num_акционные_покупки`<br>

Наименее важные признаки:

`num_маркет_актив_тек_мес`<br>
`num_ошибка_сервиса`<br>
и все признаки из `популярная_категория`

7. Пользователи в сегменте покупают в основном косметику и аксессуары, кухонную посуду и детские товары. Сегмент меньше времени проводит на сайте, но все еще пользуется акциями. Также сократилось количество маркетинговых взаимодействий с клиентом. Исходя из этого, можно проводить дальнейший анализ
